<a href="https://colab.research.google.com/github/siddengineer/YOUTUBE-PDFS/blob/main/Admission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving maharashtra_college_admissions.csv to maharashtra_college_admissions.csv


In [4]:

import pandas as pd
import folium
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error


df = pd.read_csv("maharashtra_college_admissions.csv")



city_summary = df.groupby("City").agg(
    Admissions=("Student_ID", "count"),
    Avg_Marks=("Marks", "mean")
).reset_index()

city_coords = {
    "Mumbai": [19.0760, 72.8777],
    "Pune": [18.5204, 73.8567],
    "Nagpur": [21.1458, 79.0882],
    "Nashik": [19.9975, 73.7898],
    "Aurangabad": [19.8762, 75.3433],
    "Kolhapur": [16.7050, 74.2433],
    "Solapur": [17.6599, 75.9064],
    "Satara": [17.6805, 74.0183],
    "Sangli": [16.8524, 74.5815],
    "Jalgaon": [21.0077, 75.5626],
    "Amravati": [20.9333, 77.7500],
    "Latur": [18.4088, 76.5604],
    "Nanded": [19.1383, 77.3210],
    "Akola": [20.7059, 77.0219],
    "Chandrapur": [19.9615, 79.2961],
    "Thane": [19.2183, 72.9781]
}


maha_map = folium.Map(location=[19.7515, 75.7139], zoom_start=6)
for _, row in city_summary.iterrows():
    city = row["City"]
    if city in city_coords:
        folium.CircleMarker(
            location=city_coords[city],
            radius=row["Admissions"] / 20,  # scale radius
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6,
            popup=f"{city}: {row['Admissions']} admissions, Avg Marks {row['Avg_Marks']:.2f}"
        ).add_to(maha_map)


maha_map.save("maharashtra_admissions_heatmap.html")
files.download("maharashtra_admissions_heatmap.html")

print("✅ Heatmap saved as maharashtra_admissions_heatmap.html")


yearly_data = df.groupby(["City", "Admission_Year"]).agg(
    Admissions=("Student_ID", "count"),
    Avg_Marks=("Marks", "mean"),
    Ad_Spend=("Ad_Spend", "mean"),
    Population=("Population", "mean"),
    Past_Admissions=("Past_Admissions", "mean")
).reset_index()

X = yearly_data[["Avg_Marks", "Ad_Spend", "Population", "Past_Admissions"]]
y = yearly_data["Admissions"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


future_data = yearly_data.groupby("City").mean().reset_index()
future_X = future_data[["Avg_Marks", "Ad_Spend", "Population", "Past_Admissions"]]
future_data["Predicted_Admissions_2025"] = model.predict(future_X)


future_data["ROI"] = future_data["Predicted_Admissions_2025"] / future_data["Ad_Spend"]


top_cities = future_data.sort_values(by="ROI", ascending=False).head(5)

print("\n📊 Predicted Admissions for 2025:\n")
print(future_data[["City", "Predicted_Admissions_2025"]])

print("\n💡 Top 5 Cities for Advertising ROI:\n")
print(top_cities[["City", "Predicted_Admissions_2025", "Ad_Spend", "ROI"]])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Heatmap saved as maharashtra_admissions_heatmap.html
R² Score: -0.31
MAE: 18.96

📊 Predicted Admissions for 2025:

          City  Predicted_Admissions_2025
0        Akola                      97.51
1     Amravati                      93.20
2   Aurangabad                      93.54
3   Chandrapur                     105.22
4      Jalgaon                      97.87
5     Kolhapur                      89.49
6        Latur                      81.43
7       Mumbai                      78.80
8       Nagpur                      87.44
9       Nanded                      99.41
10      Nashik                      96.57
11        Pune                     103.13
12      Sangli                      86.85
13      Satara                      98.56
14     Solapur                      76.76
15       Thane                     101.78

💡 Top 5 Cities for Advertising ROI:

          City  Predicted_Admissions_2025      Ad_Spend       ROI
3   Chandrapur                     105.22  29978.054725  0.003510